In [15]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [16]:
## Load the trained model, scaler pickle, OHE
model = load_model('model.h5')

## Load the encoder and scaler
with open('label_encoder_gender.pk1','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pk1','rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pk1','rb') as file:
    scaler = pickle.load(file)


In [17]:
## Example input data
input_data = {
    'CreditScore': 650,
    'Geography': 'France',   
    'Gender': 'Female',     
    'Age': 35,
    'Tenure': 5,
    'Balance': 60000.0,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000.0
}

## OHE on 'Geography' column
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\DEBANJALI\ANN_CLASSIFICATION\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [18]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,Female,35,5,60000.0,2,1,1,50000.0


In [19]:
## Encode categorical variables

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,0,35,5,60000.0,2,1,1,50000.0


In [20]:
## Combine OHE columns with input data
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)


# Ensure all columns expected by the scaler are present
expected_cols = scaler.feature_names_in_
input_df = input_df.reindex(columns=expected_cols, fill_value=0)

input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,0,35,5,60000.0,2,1,1,50000.0,1.0,0.0,0.0


In [21]:
## Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[ 0.04040852, -1.09997489, -0.38777267, -0.0165363 , -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [22]:
## Predict our output 
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 1s 609ms/step


array([[0.04773069]], dtype=float32)

In [23]:
prediction_probability = prediction[0][0]

if prediction_probability > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
